## Introduction

This is my solution using SQL extension on jupyter notebook for Case Study #1 of Danny Ma's 8 Weeks SQL Challenge. 

## Case Study Questions

Each of the following case study questions can be answered using a single SQL statement:

1. What is the total amount each customer spent at the restaurant?
2. How many days has each customer visited the restaurant?
3. What was the first item from the menu purchased by each customer?
4. What is the most purchased item on the menu and how many times was it purchased by all customers?
5. Which item was the most popular for each customer?
6. Which item was purchased first by the customer after they became a member?
7. Which item was purchased just before the customer became a member?
8. What is the total items and amount spent for each member before they became a member?
9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

## Getting Started

First, install SQL extension on jupyter notebook and import relevant libraries.
Then, load SQL extension

In [1]:
# !pip install ipython-sql
import sql
from sqlalchemy import create_engine


%load_ext sql
%sql sqlite://


Next, create the required tables provided- sales,members,menu

In [2]:

%%sql 

CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');

 * sqlite://
Done.
15 rows affected.


[]

In [3]:
%%sql

CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

 * sqlite://
Done.
2 rows affected.


[]

In [4]:

%%sql

CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');

 * sqlite://
Done.
3 rows affected.


[]

## Solution



#### Q1 : What is the total amount each customer spent at the restaurant? 

In [5]:
%%sql 

SELECT s.customer_id, SUM(price) AS total_sales
FROM sales AS s
JOIN menu AS m 
ON s.product_id = m.product_id
GROUP BY customer_id

 * sqlite://
Done.


customer_id,total_sales
A,76
B,74
C,36


Ans: Customer A spent $76 , 
Customer B spent $ 74 and <br>
Customer C spent $ 36 at the restaurant.

#### Q2: How many days has each customer visited the restaurant?


In [6]:
%%sql 

SELECT s.customer_id, COUNT(DISTINCT(order_date)) AS days
FROM sales AS s
GROUP BY customer_id


 * sqlite://
Done.


customer_id,days
A,4
B,6
C,2


Ans: Customer A visited the restaurant on 4 days, Customer B on 6 days and Customer C on 2 days.

#### Q3: What was the first item from the menu purchased by each customer? 

In [7]:
%%sql 

WITH ordered_sales AS
(SELECT s.customer_id, s.order_date, product_name, 
 DENSE_RANK() OVER (PARTITION BY s.customer_id 
                    ORDER BY s.order_date) AS rank
FROM sales AS s
 JOIN menu AS m
 ON s.product_id = m.product_id)

SELECT customer_id, product_name
FROM ordered_sales
WHERE rank = 1
GROUP BY customer_id, product_name;


 * sqlite://
Done.


customer_id,product_name
A,curry
A,sushi
B,curry
C,ramen


Ans: Customer A bought curry and sushi as the first item, Customer B bought curry , and Customer C ramen.

 #### Q4: What is the most purchased item on the menu and how many times was it purchased by all customers?
 

In [8]:
%%sql 
SELECT 
product_name, 
(COUNT(s.product_id)) AS Number_of_times_purchased 
FROM sales AS s
JOIN menu AS m
 ON s.product_id = m.product_id
GROUP BY s.product_id, product_name
ORDER BY Number_of_times_purchased DESC
LIMIT 1

 * sqlite://
Done.


product_name,Number_of_times_purchased
ramen,8


Ans: Ramen is the most purchased item on the menu and it was purchased 8 times by all customers.

##### Q5 : Which item was the most popular for each customer? 

In [9]:
%%sql 


WITH most_popular_item_cte AS
(SELECT s.customer_id, m.product_name, 
  COUNT(m.product_id) AS times_purchased,
  DENSE_RANK() OVER(PARTITION BY s.customer_id
  ORDER BY COUNT(s.customer_id) DESC) AS rank
FROM menu AS m
JOIN sales AS s
 ON m.product_id = s.product_id
GROUP BY s.customer_id, m.product_id, product_name)

SELECT * FROM most_popular_item_cte
WHERE rank = 1;




 * sqlite://
Done.


customer_id,product_name,times_purchased,rank
A,ramen,3,1
B,sushi,2,1
B,curry,2,1
B,ramen,2,1
C,ramen,3,1


Ans: The most popular item for Customer A and C is ramen. For customer B, he enjoys ramen, sushi and curry all the same.

#### Q6. Which item was purchased first by the customer after they became a member?

In [10]:
%%sql 
WITH member_sales_cte AS 
(SELECT s.customer_id, m.join_date, s.order_date,   s.product_id,
         DENSE_RANK() OVER(PARTITION BY s.customer_id
  ORDER BY s.order_date) AS rank
     FROM sales AS s
 JOIN members AS m
  ON s.customer_id = m.customer_id
 WHERE s.order_date >= m.join_date)

SELECT s.customer_id,s.order_date,  m.product_name 
FROM member_sales_cte AS s
JOIN menu as m
on s.product_id = m.product_id
where s.rank = 1
order by s.customer_id


 * sqlite://
Done.


customer_id,order_date,product_name
A,2021-01-07,curry
B,2021-01-11,sushi


Ans: <br>
    After Customer A became a member, curry was the first item purchased. <br>
    After Customer B became a member, sushi was the first item purchased.


#### Q7. Which item was purchased just before the customer became a member?

In [11]:
%%sql 
WITH member_sales_cte AS 
(SELECT s.customer_id, m.join_date, s.order_date,   s.product_id,
         DENSE_RANK() OVER(PARTITION BY s.customer_id
  ORDER BY s.order_date DESC) AS rank
     FROM sales AS s
 JOIN members AS m
  ON s.customer_id = m.customer_id
 WHERE s.order_date < m.join_date)

SELECT s.customer_id,s.order_date,  m.product_name 
FROM member_sales_cte AS s
JOIN menu as m
on s.product_id = m.product_id
where s.rank = 1
order by s.customer_id

 * sqlite://
Done.


customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-04,sushi


Ans: Customer A purchased sushi and curry while Customer B purchased sushi just before they became members.

#### Q8. What is the total items and amount spent for each member before they became a member?

In [12]:
%%sql 


SELECT s.customer_id,  SUM(mm.price) AS total_sales, COUNT(DISTINCT s.product_id) AS nunique
FROM sales AS s
JOIN members AS m
 ON s.customer_id = m.customer_id 
JOIN menu AS mm
 ON s.product_id = mm.product_id
     WHERE s.order_date < m.join_date

GROUP BY s.customer_id;

 * sqlite://
Done.


customer_id,total_sales,nunique
A,25,2
B,40,2


Ans: Customer A spent $25 on 2 items while Customer B spent $40 on 2 items before they became members.

#### Q9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier — how many points would each customer have?

In [13]:
%%sql
WITH points_cte AS
(SELECT menu.product_id, menu.product_name, 
CASE WHEN menu.product_name = 'sushi' THEN menu.price*20 
ELSE price*10 END AS points
from menu)

SELECT customer_id,SUM(points_cte.points) AS total_points
FROM points_cte JOIN sales
on points_cte.product_id = sales.product_id
GROUP BY sales.customer_id




 * sqlite://
Done.


customer_id,total_points
A,860
B,940
C,360


With the multiplier point system in place, Customer A, B and C would have 860 points, 940 points and 360 points respectively.

#### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [14]:
%%sql
WITH days_cte AS
(SELECT members.customer_id, members.join_date,sales.order_date,sales.product_id,
julianday(sales.order_date)-julianday(members.join_date) AS days_diff
from sales 
INNER JOIN members on sales.customer_id = members.customer_id)


SELECT SUM(CASE WHEN days_cte.product_id =1 THEN menu.price*20
        WHEN days_diff BETWEEN 0 and 6 and days_diff >=0 THEN menu.price*20
 WHEN days_diff < 0 THEN menu.price * 10
WHEN days_cte.product_id =1 THEN menu.price*20
WHEN days_cte.order_date >= "2021-02-01" THEN menu.price*0

ELSE menu.price*10 END) AS total_points, days_cte.product_id,  days_cte.customer_id,days_cte.order_date, days_cte.join_date, days_cte.days_diff
FROM days_cte
LEFT JOIN MENU on days_cte.product_id = menu.product_id
GROUP BY customer_id




 * sqlite://
Done.


total_points,product_id,customer_id,order_date,join_date,days_diff
1370,1,A,2021-01-01,2021-01-07,-6.0
820,2,B,2021-01-01,2021-01-09,-8.0


Ans: Customer A has 1370 points and Customer B has 820 at the end of January.